<a href="https://colab.research.google.com/github/socialx-analytics/bi-11-sept-25/blob/main/002_scraping_inflasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import requests
import pandas as pd
import time

from bs4 import BeautifulSoup
from google.colab import files

In [ ]:
# Function to scrape BI Rate data
def scrape_bi_rate():
    # Helper function to extract data from a single page
    def scrape_page(soup):
        table = soup.find('table', {'class': 'table table-striped table-no-bordered table-lg'})
        data = []
        for row in table.find_all('tr')[1:]:  # Skip header row
            cols = row.find_all('td')
            if cols:
                no = cols[0].text.strip()
                date = cols[1].text.strip()
                bi_rate = cols[2].text.strip()
                data.append({'No': no, 'Tanggal': date, 'BI-Rate': bi_rate})
        return data

    url = "https://www.bi.go.id/id/statistik/indikator/bi-rate.aspx"
    session = requests.Session()
    all_data = []

    # Loop through 11 pages of data
    for page in range(1, 12):
        print(f"Scraping BI-Rate page {page}...")
        if page == 1:
            response = session.get(url)
        else:
            # Prepare payload for POST request to navigate pages
            payload = {
                "__EVENTTARGET": f"ctl00$ctl54$g_78f62327_0ad4_4bb8_b958_a315eccecc27$ctl00$DataPagerBI7DRR$ctl01$ctl{page-1:02d}",
                "__EVENTARGUMENT": "",
                "__LASTFOCUS": "",
                "__VIEWSTATE": soup.find('input', {'name': '__VIEWSTATE'})['value'],
                "__VIEWSTATEGENERATOR": soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                "__EVENTVALIDATION": soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
            }
            response = session.post(url, data=payload)

        soup = BeautifulSoup(response.content, 'html.parser')
        page_data = scrape_page(soup)
        all_data.extend(page_data)
        time.sleep(2)  # Wait between requests to be polite

    return pd.DataFrame(all_data)

# Function to scrape inflation data
def scrape_inflasi():
    # Helper function to extract data from a single page
    def scrape_page(soup):
        table = soup.find('table', {'class': 'table table-striped table-no-bordered table-lg'})
        data = []
        for row in table.find_all('tr')[1:]:  # Skip header row
            cols = row.find_all('td')
            if cols:
                date = cols[0].text.strip()
                inflation = cols[1].text.strip()
                data.append({'Tanggal': date, 'Data Inflasi': inflation})
        return data

    url = "https://www.bi.go.id/id/statistik/indikator/data-inflasi.aspx"
    session = requests.Session()
    all_data = []

    # Loop through 26 pages of data
    for page in range(1, 27):
        print(f"Scraping Inflation page {page}...")
        if page == 1:
            response = session.get(url)
        else:
            # Prepare payload for POST request to navigate pages
            payload = {
                "__EVENTTARGET": "ctl00$ctl54$g_1f0a867d_90e9_4a92_b1c8_de34738fc4f1$ctl00$DataPagerDataInflasi$ctl02$ctl00",
                "__EVENTARGUMENT": "",
                "__LASTFOCUS": "",
                "__VIEWSTATE": soup.find('input', {'name': '__VIEWSTATE'})['value'],
                "__VIEWSTATEGENERATOR": soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                "__EVENTVALIDATION": soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
            }
            response = session.post(url, data=payload)

        soup = BeautifulSoup(response.content, 'html.parser')
        page_data = scrape_page(soup)
        all_data.extend(page_data)
        time.sleep(2)  # Wait between requests to be polite

    return pd.DataFrame(all_data)

In [ ]:
# Scrape BI Rate data and store in DataFrame
df_birate = scrape_bi_rate()
df_birate

In [ ]:
# Scrape inflation data and store in DataFrame
df_inflasi = scrape_inflasi()
df_inflasi

In [ ]:
# Save as CSV
df_inflasi.to_csv('inflasi.csv', index=False)
df_birate.to_csv('birate.csv', index=False)

# Download CSV files
files.download('inflasi.csv')
files.download('birate.csv')